In [ ]:
import torch
from PIL import Image
import numpy as np
import torchvision
import torchvision.transforms as T
from torchvision.transforms import ToTensor, Normalize , Compose
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# data_set = np.memmap(r'F:\Video_set\All_data_set.npy',mode='r',shape=(8991,100,100,3))
# x_np = torch.from_numpy(data_set)
# x_np.shape

In [ ]:
data_set = np.load(r'F:\Video_set\Sample_data_set.npy',mmap_mode="r+")
#data_set = np.load(r'F:\Video100.npy')
#np.random.shuffle(data_set)
#data_set = data_set.memmap.astype(float)
x_np = torch.from_numpy(data_set)
x_np.shape

In [ ]:
x_np.shape
x_np.type()

In [ ]:
x_np[0]

In [ ]:
x_img = x_np.reshape((849,60,100,100,3))
#x_img = x_np.reshape((99,60,100,100,3))

In [ ]:
#x_np = x_np.permute(0,3,2,1)
a = x_np.numpy()
img = a
img.shape
plt.imshow(img[21323])

In [ ]:
x_img[1,2].shape


In [ ]:
img = x_img[1,40]
img.shape
plt.imshow(img)

In [ ]:

image_size = 100
batch_size = 1
stats = 1/255

In [ ]:
x_np[0].type()

In [ ]:
#x_np = x_np.memmap.astype(float)

In [ ]:
# x_np = x_np.float()
# x_np = x_np/255
# x_np.shape

In [ ]:
x_np = x_np.permute(0,3,1,2)
x_img = x_np.reshape((849,3,60,100,100))
x_img.shape

In [ ]:
x_img[0]

In [ ]:
x_np.shape

In [ ]:
x_img.shape

In [ ]:
x_data_set = DataLoader(x_img,batch_size,shuffle=True)

In [ ]:
x_data_set

In [ ]:
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def denorm(img_tensors):
    out = img_tensors
    return out.clamp(0,1)

In [ ]:
def show_images(images, nmax=60):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(images.detach()[:nmax], nrow=10).permute(1,2,0))

def show_batch(dl, nmax=60):
    for image in dl:
        for images in image:
            images = images.reshape(60,3,100,100)
            show_images(images, nmax)
            print(images.shape)
            break
        break

In [ ]:
show_batch(x_data_set)


In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
x_data_set = DeviceDataLoader(x_data_set, device)


In [ ]:
import torch.nn as nn



In [ ]:
discriminator = nn.Sequential(
    #in: 3 x 60 x 100 x 100 
    nn.Conv3d(3, 64, kernel_size=(4,5,5), stride=(2,5,5), padding=1, bias=False),
    nn.BatchNorm3d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out 64 x 30 x 50 x 50
    nn.Conv3d(64, 128, kernel_size=(4,4,4), stride=(2,2,2), padding=1, bias=False),
    nn.BatchNorm3d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # 128 x 25 x 25
    nn.Conv3d(128, 256, kernel_size=(4,4,4), stride=(2,2,2), padding=1, bias=False),
    nn.BatchNorm3d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv3d(256, 512, kernel_size=(4,4,4), stride=(2,2,2), padding=1, bias=False),
    nn.BatchNorm3d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv3d(512, 1, kernel_size=(3,2,2), stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid())
discriminator = to_device(discriminator, device)


In [ ]:
# discriminator = nn.Sequential(
#     #in: 3 x 60 x 100 x 100 
#     nn.Conv3d(3, 64, kernel_size=(4,5,5), stride=2, padding=1, bias=False),
#     nn.BatchNorm3d(64),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out 64 x 30 x 50 x 50
#     nn.Conv3d(64, 128, kernel_size=(4,4,4), stride=2, padding=1, bias=False),
#     nn.BatchNorm3d(128),
#     nn.LeakyReLU(0.2, inplace=True),
#     # 128 x 25 x 25
#     nn.Conv3d(128, 256, kernel_size=(3,4,4), stride=2, padding=1, bias=False),
#     nn.BatchNorm3d(256),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out: 256 x 8 x 8

#     nn.Conv3d(256, 512, kernel_size=(4,4,4), stride=1, padding=1, bias=False),
#     nn.BatchNorm3d(512),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out: 512 x 4 x 4

#     nn.Conv3d(512, 1, kernel_size=(4,4,4), stride=1, padding=0, bias=False),
#     # out: 1 x 1 x 1

#     nn.Flatten(),
#     nn.Sigmoid())
# discriminator = to_device(discriminator, device)


In [ ]:
# discriminator = nn.Sequential(
#     nn.Conv3d(3, 128, kernel_size=(4,4,4), stride=4, padding=0, bias=False),
#     nn.BatchNorm3d(128),
#     nn.LeakyReLU(0.2, inplace=True),
    
#     nn.Conv3d(128, 256, kernel_size=(5,5,5), stride=4, padding=1, bias=False),
#     nn.BatchNorm3d(256),
#     nn.LeakyReLU(0.2, inplace=True),
#     nn.Conv3d(256, 512, kernel_size=(4,6,6), stride=1, padding=1, bias=False),
#     nn.BatchNorm3d(512),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out: 512 x 4 x 4
#     nn.ConvTranspose3d(512, 60, kernel_size=(4,4,4), stride=1, padding=(0,0,0), bias=False),
#     nn.Flatten(),
#     nn.Sigmoid())
# discriminator = to_device(discriminator, device)

In [ ]:
latent_size = 64


In [ ]:
generator = nn.Sequential(
    #input 3x100x100
#     nn.Conv3d(3,64,kernel_size=(3,3,3),bias=False),
#     nn.BatchNorm2d(64),
#     nn.ReLU(True),
    
#     nn.Conv3d(64,128,kernel_size=(3,3,3),bias=False),
#     nn.BatchNorm2d(128),
#     nn.ReLU(True),
#     nn.Conv3d(128,512,kernel_size=(3,3,3),bias=False),
#     nn.BatchNorm2d(512),
#     nn.ReLU(True),
    nn.Conv3d(3, 128, kernel_size=(4,4,4), stride=4, padding=0, bias=False),
    nn.BatchNorm3d(128),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv3d(128, 256, kernel_size=(5,5,5), stride=4, padding=1, bias=False),
    nn.BatchNorm3d(256),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv3d(256, 512, kernel_size=(4,6,6), stride=1, padding=1, bias=False),
    nn.BatchNorm3d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4
    nn.ConvTranspose3d(512, 60, kernel_size=(4,4,4), stride=1, padding=(0,0,0), bias=False),


    nn.ConvTranspose3d(60, 256, kernel_size=(5,5,5), stride=2, padding=(0,1,1), bias=False),
    nn.BatchNorm3d(256),
    nn.ReLU(True),
    
    nn.ConvTranspose3d(256, 128, kernel_size=(4,4,4), stride=2, padding=(0,1,1), bias=False),
    nn.BatchNorm3d(128),
    nn.ReLU(True),

    nn.ConvTranspose3d(128, 64, kernel_size=(4,4,4), stride=(1,2,2), padding=(1,1,1), bias=False),
    nn.BatchNorm3d(64),
    nn.ReLU(True),
    nn.ConvTranspose3d(64, 3, kernel_size=(4,4,4), stride=2, padding=(4,3,3), bias=False),
    nn.Tanh()    
    
)
#xb = torch.randn(60, latent_size, 1, 1) # random latent tensors

xb = torch.unsqueeze(x_img[0],0)
fake_images = generator(xb.float())
fake_images.shape
#show_images(denorm(fake_images))
#print(fake_images.shape)

In [ ]:
xb.shape

In [ ]:
#generator = to_device(generator, device)


In [ ]:
xb = torch.unsqueeze(x_img[1],0)
#xb = x_img
x_img[0].shape
fake_images = generator(xb.float())


In [ ]:
xb.shape

In [ ]:

fake_images[0].shape
fake_images.shape
fake_images = fake_images.reshape(1,60,3,100,100)
show_images(denorm(fake_images[0]))
print(fake_images[0].shape)

In [ ]:
# generator = nn.Sequential(
#     #input 3x100x100
# #     nn.Conv3d(3,64,kernel_size=(3,3,3),bias=False),
# #     nn.BatchNorm2d(64),
# #     nn.ReLU(True),
    
# #     nn.Conv3d(64,128,kernel_size=(3,3,3),bias=False),
# #     nn.BatchNorm2d(128),
# #     nn.ReLU(True),
# #     nn.Conv3d(128,512,kernel_size=(3,3,3),bias=False),
# #     nn.BatchNorm2d(512),
# #     nn.ReLU(True),
#     nn.Conv3d(3, 64, kernel_size=[2,4,4],stride=[1,1,1], padding=[3,1,1],bias=False),
#     nn.BatchNorm2d(64),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out 64x 50 x 50
#     nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(128),
#     nn.LeakyReLU(0.2, inplace=True),
#     # 128 x 25 x 25
#     nn.Conv3d(128, 256, kernel_size=4, stride=5, padding=1, bias=False),
#     nn.BatchNorm2d(256),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out: 256 x 8 x 8
#     nn.Conv3d(256, 512, kernel_size=4, stride=1, padding=1, bias=False),
#     nn.BatchNorm2d(512),
#     nn.LeakyReLU(0.2, inplace=True),
#     # out: 512 x 4 x 4
#     nn.Conv3d(512, 60, kernel_size=4, stride=1, padding=0, bias=False),


#     nn.ConvTranspose3d(60, 256, kernel_size=(5,5,5), stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(121),
#     nn.ReLU(True),
    
#     nn.ConvTranspose3d(256, 128, kernel_size=(4,4,4), stride=2, padding=0, bias=False),
#     nn.BatchNorm2d(244),
#     nn.ReLU(True),

#     nn.ConvTranspose3d(128, 64, kernel_size=(4,4,4), stride=2, padding=0, bias=False),
#     nn.BatchNorm2d(490 ),
#     nn.ReLU(True),
#     nn.ConvTranspose3d(64, 3, kernel_size=(4,4,4), stride=2, padding=1, bias=False),
#     nn.Tanh()    
    
# )
# #xb = torch.randn(60, latent_size, 1, 1) # random latent tensors
# xb = x_img[0]
# fake_images = generator(xb.float())
# #show_images(denorm(fake_images))
# print(fake_images.shape)


In [ ]:
# generator = nn.Sequential(
#     # in: latent_size x 1 x 1

#     nn.ConvTranspose2d(latent_size, 512, kernel_size=5, stride=1, padding=0, bias=False),
#     nn.BatchNorm2d(512),
#     nn.ReLU(True),
#     # out: 512 x 5 x 5

#     nn.ConvTranspose2d(512, 256, kernel_size=5, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(256),
#     nn.ReLU(True),
    
#     nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=0, bias=False),
#     nn.BatchNorm2d(128),
#     nn.ReLU(True),

#     nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=0, bias=False),
#     nn.BatchNorm2d(64),
#     nn.ReLU(True),
#     nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.Tanh()
# )
# xb = torch.randn(60, latent_size, 1, 1) # random latent tensors



In [ ]:
def train_discriminator(real_images, opt_d):
    # Clear discriminator gradients
    opt_d.zero_grad()
    # Pass real images through discriminator
    real_preds = discriminator(real_images)
    real_targets = torch.ones(real_images.size(0), 1, device=device)
    real_loss = F.binary_cross_entropy(real_preds, real_targets)
    real_score = torch.mean(real_preds).item()
    
    # Generate fake images
    #latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(torch.unsqueeze(real_images[0],0))

    # Pass fake images through discriminator
    fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
    fake_preds = discriminator(fake_images)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)
    fake_score = torch.mean(fake_preds).item()

    # Update discriminator weights
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score

In [ ]:
def train_generator(real_images,opt_g):
    # Clear generator gradients
    opt_g.zero_grad()
    
    # Generate fake images
#     latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
#     fake_images = generator(latent)
    fake_images = generator(torch.unsqueeze(real_images,0))
    # Try to fool the discriminator
    preds = discriminator(fake_images)
    targets = torch.ones(batch_size, 1, device=device)
    loss = F.binary_cross_entropy(preds, targets)
    
    # Update generator weights
    loss.backward()
    opt_g.step()
    
    return loss.item()

In [ ]:
from torchvision.utils import save_image


In [ ]:
import os
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

In [ ]:
def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors.float())
    fake_images = fake_images.reshape(1,60,3,100,100)
    print(fake_images[0].shape)

    fake_images = denorm(fake_images)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(fake_images[0], os.path.join(sample_dir, fake_fname), nrow=10)
    print('Saving', fake_fname)
    if show:
        for image in fake_images:
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.set_xticks([]); ax.set_yticks([])
            fake_images.shape
            ax.imshow(make_grid(image.cpu().detach(), nrow=10).permute(1, 2, 0))
#         #ax.imshow(make_grid(denomrfake_images.cpu().detach(), nrow=6).permute(2, 1, 0))

In [ ]:
# fixed_latent = torch.randn(60, latent_size, 1, 1, device=device)
# save_samples(0, fixed_latent)
fake_images = torch.unsqueeze(x_img[20],0)
fake_images.shape
fake_images = fake_images.reshape(1,3,60,100,100)
generator = to_device(generator, 'cpu')
save_samples(0,fake_images)

In [ ]:
generator = to_device(generator, device)

In [ ]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [ ]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    # Create optimizers
    opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.9))
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.9))

    for epoch in range(epochs):
        for real_images in tqdm(x_data_set):
            # Train discriminator
            real_images = real_images.float()
            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            # Train generator
            loss_g = train_generator(real_images[0],opt_g)
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # Save generated images
        save_samples(epoch+start_idx, torch.unsqueeze(real_images[0],0), show=False)
    
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
lr = 0.0002
epochs = 50

In [ ]:
history = fit(epochs, lr)


In [ ]:
# # Save the model checkpoints 
# torch.save(generator.state_dict(), 'generated/G.pth')
# torch.save(discriminator.state_dict(), 'generated/D.pth')
torch.save(generator, 'generated/Gen.pth')
torch.save(discriminator, 'generated/Dis.pth')

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history


In [ ]:
#torch.cuda.empty_cache()


In [ ]:
generator = to_device(generator,'cpu')

In [ ]:
# fake_images = generator()
# show_images(denorm(fake_images))
# print(fake_images.shape)